In [1]:
import imp
import timeit
import torch
import numpy as np
import ops
from utils import lattice

In [2]:
imp.reload(ops)

<module 'ops' from '/Users/theissjd/Documents/Berkeley/code/rf_pool/rf_pool/ops.py'>

In [3]:
# test ops.average_pool
rf_u = torch.rand(1,10,16)
torch.random.manual_seed(0)
out_shape = (1,10,4,4)
h_mean, h_sample, p_mean, p_sample = ops.average_pool(rf_u, out_shape)
# manually perform average_pool
probs = torch.div(rf_u, rf_u.shape[-1])
assert torch.all(torch.eq(h_mean, torch.reshape(probs, out_shape)))
torch.random.manual_seed(0)
samples = torch.distributions.Multinomial(probs=torch.softmax(probs, -1)).sample()
assert torch.all(torch.eq(h_sample, torch.reshape(samples, out_shape)))
p_probs = torch.mul(probs, samples)
assert torch.all(torch.eq(p_mean, torch.reshape(p_probs, out_shape)))
p_samples = samples.clone()
assert torch.all(torch.eq(p_sample, torch.reshape(p_samples, out_shape)))

In [4]:
# test ops.div_norm_pool
rf_u = torch.rand(1,10,16)
torch.random.manual_seed(0)
out_shape = (1,10,4,4)
n = 2.
sigma = 0.5
h_mean, h_sample, p_mean, p_sample = ops.div_norm_pool(rf_u, out_shape, n=n, sigma=sigma)
# manually perform div_norm_pool
rf_u_n = torch.pow(rf_u, n)
sigma_n = torch.pow(torch.as_tensor(sigma, dtype=rf_u.dtype), n)
probs = torch.div(rf_u_n, sigma_n + torch.sum(rf_u_n, dim=-1, keepdim=True))
assert torch.all(torch.eq(h_mean, torch.reshape(probs, out_shape)))
torch.random.manual_seed(0)
samples = torch.distributions.Multinomial(probs=torch.softmax(probs, -1)).sample()
assert torch.all(torch.eq(h_sample, torch.reshape(samples, out_shape)))
p_probs = torch.reshape(torch.mul(probs, samples), out_shape)
assert torch.all(torch.eq(p_mean, p_probs))
p_samples = torch.reshape(samples, out_shape)
assert torch.all(torch.eq(p_sample, p_samples))

In [5]:
# test ops.prob_max_pool
rf_u = torch.rand(1,10,16)
torch.random.manual_seed(0)
out_shape = (1,10,4,4)
h_mean, h_sample, p_mean, p_sample = ops.prob_max_pool(rf_u, out_shape)
# manually perform prob_max_pool
rf_u_0 = torch.cat([rf_u, torch.zeros(1,10,1)], -1)
rf_u_0_softmax = torch.softmax(rf_u_0, -1)
probs = torch.reshape(rf_u_0_softmax[:,:,:-1], out_shape)
assert torch.all(torch.eq(h_mean, probs))
torch.random.manual_seed(0)
samples = torch.distributions.Multinomial(probs=rf_u_0_softmax).sample()
samples = torch.reshape(samples[:,:,:-1], out_shape)
assert torch.all(torch.eq(h_sample, samples))
p_probs = torch.mul(torch.reshape(1. - rf_u_0_softmax[:,:,-1], (1,10,1,1)), samples)
assert torch.all(torch.eq(p_mean, p_probs))

In [6]:
# test ops.stochastic_max_pool
rf_u = torch.rand(1,10,16)
torch.random.manual_seed(0)
out_shape = (1,10,4,4)
h_mean, h_sample, p_mean, p_sample = ops.stochastic_max_pool(rf_u, out_shape)
# manually perform prob_max_pool
rf_u_softmax = torch.softmax(rf_u, -1)
probs = torch.reshape(rf_u_softmax, out_shape)
assert torch.all(torch.eq(h_mean, probs))
torch.random.manual_seed(0)
samples = torch.distributions.Multinomial(probs=rf_u_softmax).sample()
samples = torch.reshape(samples, out_shape)
assert torch.all(torch.eq(h_sample, samples))
p_probs = torch.mul(probs, samples)
assert torch.all(torch.eq(p_mean, p_probs))
p_samples = samples.clone()
assert torch.all(torch.eq(p_sample, p_samples))

In [7]:
# test ops.sum_pool
rf_u = torch.rand(1,10,16)
torch.random.manual_seed(0)
out_shape = (1,10,4,4)
h_mean, h_sample, p_mean, p_sample = ops.sum_pool(rf_u, out_shape)
# manually perform sum_pool
probs = torch.reshape(rf_u, out_shape)
assert torch.all(torch.eq(h_mean, probs))
torch.random.manual_seed(0)
samples = torch.zeros_like(rf_u)
samples.add_(torch.sum(rf_u, dim=-1, keepdim=True))
samples = torch.reshape(samples, out_shape)
sampled_pos = torch.reshape(torch.distributions.Multinomial(probs=torch.softmax(rf_u, -1)).sample(), out_shape)
samples = torch.mul(samples, sampled_pos)
assert torch.all(torch.eq(h_sample, samples))
p_probs = torch.mul(h_mean, sampled_pos)
assert torch.all(torch.eq(p_mean, p_probs))
p_samples = torch.mul(h_sample, sampled_pos)
assert torch.all(torch.eq(p_sample, p_samples))

In [8]:
# test ops.rf_pool defaults #TODO: update with p_mean, p_sample
u = torch.rand(1,10,16,16)
torch.random.manual_seed(0)
h_mean, h_sample, p_mean, p_sample = ops.rf_pool(u)
# manually perform ops.rf_pool with defaults
torch.random.manual_seed(0)
b = []
for r in range(2):
    for c in range(2):
        b.append(u[:,:,r::2,c::2].unsqueeze(-1))
b = torch.cat(b, -1)
tmp_probs, tmp_samples, tmp_p_probs, tmp_p_samples = ops.prob_max_pool(b, b.shape)
probs = torch.zeros_like(u)
samples = torch.zeros_like(u)
p_probs = torch.zeros(1,10,8,8)
p_samples = torch.zeros(1,10,8,8)
for r in range(2):
    for c in range(2):
        probs[:,:,r::2,c::2] = tmp_probs[:,:,:,:,r*2+c]
        samples[:,:,r::2,c::2] = tmp_samples[:,:,:,:,r*2+c]
        p_probs = torch.max(p_probs, tmp_p_probs[:,:,:,:,r*2+c])
        p_samples = torch.max(p_samples, tmp_p_samples[:,:,:,:,r*2+c])
assert torch.all(torch.eq(h_mean, probs))
assert torch.all(torch.eq(h_sample, samples))
assert torch.all(torch.eq(p_mean, p_probs))
assert torch.all(torch.eq(p_sample, p_samples))

In [9]:
# test ops.rf_pool top-down, div_norm, block_size, pool_args #TODO: update with p_mean, p_sample
u = torch.rand(1,10,16,16)
t = torch.rand(1,10,4,4)
torch.random.manual_seed(0)
pool_type = 'div_norm'
block_size = (4,4)
pool_args = [1., 0.5]
h_mean, h_sample, p_mean, p_sample = ops.rf_pool(u, t, pool_type=pool_type, 
                                             block_size=block_size, pool_args=pool_args)
# manually perform ops.rf_pool with top-down, div_norm, block_size, pool_args
b = []
for r in range(block_size[0]):
    for c in range(block_size[1]):
        u[:,:,r::block_size[0],c::block_size[1]].add_(t)
        b.append(u[:,:,r::block_size[0],c::block_size[1]].unsqueeze(-1))
b = torch.cat(b, -1)
# get tmp_probs, tmp_samples from div_norm_pool using pool_args
torch.random.manual_seed(0)
tmp_probs, tmp_samples, tmp_p_probs, tmp_p_samples = ops.div_norm_pool(b, b.shape, *pool_args)
probs = torch.zeros_like(u)
samples = torch.zeros_like(u)
# get probs, samples, p_probs, p_samples
p_probs = torch.zeros_like(t)
p_samples = torch.zeros_like(t)
for r in range(block_size[0]):
    for c in range(block_size[1]):
        probs[:,:,r::block_size[0],c::block_size[1]] = tmp_probs[:,:,:,:,r*block_size[0]+c]
        samples[:,:,r::block_size[0],c::block_size[1]] = tmp_samples[:,:,:,:,r*block_size[0]+c]
        p_probs = torch.max(p_probs, tmp_p_probs[:,:,:,:,r*block_size[0]+c])
        p_samples = torch.max(p_samples, tmp_p_samples[:,:,:,:,r*block_size[0]+c])
assert torch.all(torch.eq(h_mean, probs))
assert torch.all(torch.eq(h_sample, samples))
assert torch.all(torch.eq(p_mean, p_probs))
assert torch.all(torch.eq(p_sample, p_samples))

In [10]:
# test ops.rf_pool kernels with batch_size > 1
mu, sigma = lattice.init_uniform_lattice((8,8), 4, 4, 1.)
delta_mu = torch.rand(2,1,16,2)
delta_sigma = torch.rand(2,1,16,1)
mu = mu + delta_mu
sigma = sigma + delta_sigma
rfs = lattice.gaussian_kernel_lattice(mu, sigma, (16,16))
u = torch.rand(2,10,16,16)
torch.random.manual_seed(0)
h_mean, h_sample, p_mean, p_sample = ops.rf_pool(u, rfs=rfs, pool_type='sum', block_size=(1,1))
# manually perform pooling with gaussian rfs
probs = torch.zeros_like(u)
samples = torch.zeros_like(u)
p_probs = torch.zeros_like(u)
p_samples = torch.zeros_like(u)
u.unsqueeze_(2)
rfs = torch.add(torch.zeros_like(u), rfs)
g_u = torch.mul(u, rfs).permute(2,0,1,3,4)
rf_index = torch.gt(rfs, 1e-5).permute(2,0,1,3,4)
rf_u = torch.zeros_like(g_u) - np.inf
rf_u[rf_index] = g_u[rf_index]
torch.random.manual_seed(0)
probs, samples, p_probs, p_samples = ops.sum_pool(rf_u.flatten(-2), rf_u.shape)
probs = torch.max(probs, 0)[0]
samples = torch.max(samples, 0)[0]
p_probs = torch.max(p_probs, 0)[0]
p_samples = torch.max(p_samples, 0)[0]
assert torch.all(torch.eq(h_mean, probs))
assert torch.all(torch.eq(h_sample, samples))
assert torch.all(torch.eq(p_mean, p_probs))
assert torch.all(torch.eq(p_sample, p_samples))

In [11]:
# test time taken for indexing or adding mask
s = timeit.default_timer()
rfs = torch.add(torch.zeros_like(u), rfs)
g_u = torch.mul(u, rfs).permute(2,0,1,3,4)
rf_index = torch.gt(rfs, 1e-5).permute(2,0,1,3,4)
rf_u0 = torch.zeros_like(g_u) - np.inf
rf_u0[rf_index] = g_u[rf_index]
e = timeit.default_timer()
print('index:', e - s)

s = timeit.default_timer()
rfs = torch.add(torch.zeros_like(u), rfs)
g_u = torch.mul(u, rfs).permute(2,0,1,3,4)
# create rf_mask of 0s at rf and -inf elsewhere (optimized)
thr = 1e-5
rf_mask = torch.as_tensor(torch.le(rfs.permute(2,0,1,3,4), thr), dtype=g_u.dtype)
rf_mask = -1. * torch.exp(np.inf * (2. * rf_mask - 1.))
rf_u1 = torch.add(g_u, rf_mask)
e = timeit.default_timer()
print('add:', e - s)
assert torch.all(torch.eq(rf_u0, rf_u1))

index: 0.030836334044579417
add: 0.0014781790086999536
